<a href="https://colab.research.google.com/github/ziedjouini/new_projet/blob/main/Ensemble_Learning_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
from io import BytesIO
import urllib.request as urllib2
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split


from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import VotingClassifier

from sklearn.preprocessing import StandardScaler

# **Practicing ensemble learning**
Download the following [Spotify dataset](https://raw.githubusercontent.com/murpi/wilddata/master/quests/spotify.zip).

Execute the following tasks:
* Build a classification or regression model with this dataset
* Evaluate the performance of your  model using accuracy score, RMSE or R_squared
* Apply 3 ensemble techniques of your choice on the dataset
* Compare the obtained results with your individual model


In [ ]:
# df reading
lien = "https://raw.githubusercontent.com/murpi/wilddata/master/quests/spotify.zip"
r = urllib2.urlopen(lien).read()
file = ZipFile(BytesIO(r))
df_csv = file.open("SpotifyFeatures.csv")
df = pd.read_csv(df_csv)
df =df.sample(n = 100000)

In [ ]:
df

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
86614,R&B,James Morrison,Wonderful World,7EfHlhxyYJ5pfh0H34j7oe,42,0.05970,0.489,209787,0.70200,0.00000,C#,0.1000,-5.205,Major,0.0277,78.995,4/4,0.6440
91560,Hip-Hop,Darci,Sick Too,2TU0CXXtDUcncMREf32h9v,49,0.02760,0.484,169286,0.54100,0.00000,G,0.1380,-8.701,Major,0.0370,139.981,4/4,0.1370
160626,Reggaeton,Mark B.,Muah,4JOOh7dX5Slq6BM3hzJ7VA,39,0.03550,0.707,209288,0.84200,0.00007,B,0.0793,-3.004,Minor,0.1720,101.984,4/4,0.7330
210855,World,Hiroshi Yoshimura,Ice Copy,4YleYbTBE8XmKs5fhuu06c,32,0.98700,0.270,175773,0.00157,0.88200,C#,0.1090,-31.359,Major,0.0438,62.892,3/4,0.0798
56580,R&B,Asiahn,Curiosity,7cGzEpKCeLvODoxLFy7ntV,55,0.44500,0.457,234802,0.42500,0.00000,E,0.0714,-10.500,Minor,0.2430,201.472,4/4,0.4980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42466,Folk,Henry Green,Electric Feel (Kygo Remix),0toJeVTW2hHX9qYVmdOpBs,59,0.23800,0.709,390649,0.36900,0.16800,E,0.0704,-13.104,Minor,0.0488,102.010,3/4,0.3730
206348,Soundtrack,Frederik Wiedmann,I See You,6r4F9uzW3VddeHeOW39nHG,25,0.95500,0.354,56000,0.10600,0.97500,C#,0.1130,-22.190,Major,0.0387,139.863,4/4,0.0322
166062,Rock,Imagine Dragons,Thunder,1zB4vmk8tFRmM9UULNzbLB,88,0.00671,0.605,187147,0.82200,0.13400,C,0.1470,-4.833,Major,0.0438,167.997,4/4,0.2880
17027,Dance,Kelly Clarkson,Catch My Breath,5qxPabNDnJQ551Tf4v6k3j,54,0.02510,0.692,250387,0.90800,0.00000,A,0.0933,-5.364,Major,0.0677,124.042,4/4,0.5130


In [ ]:
print( "Le nombre Nan dans le dataFrame est de ",df.isnull().sum().sum())

Le nombre Nan dans le dataFrame est de  0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 86614 to 199007
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   genre             100000 non-null  object 
 1   artist_name       100000 non-null  object 
 2   track_name        100000 non-null  object 
 3   track_id          100000 non-null  object 
 4   popularity        100000 non-null  int64  
 5   acousticness      100000 non-null  float64
 6   danceability      100000 non-null  float64
 7   duration_ms       100000 non-null  int64  
 8   energy            100000 non-null  float64
 9   instrumentalness  100000 non-null  float64
 10  key               100000 non-null  object 
 11  liveness          100000 non-null  float64
 12  loudness          100000 non-null  float64
 13  mode              100000 non-null  object 
 14  speechiness       100000 non-null  float64
 15  tempo             100000 non-null  float64
 16  time_signature  

In [ ]:
txt = "Le nombre de valeur différente de track_name (" + str(len(df["track_name"].unique())) + "), track_id et artist_name (" + str(len(df["artist_name"].unique())) +") étant tres importante on supprime ces colones "

print(txt)

Le nombre de valeur différente de track_name (76237), track_id et artist_name (11466) étant tres importante on supprime ces colones 


In [ ]:
df2 = df.drop(["artist_name", 	"track_name", 	"track_id"], axis = 1)
df2

,genre,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
86614,R&B,42,0.05970,0.489,209787,0.70200,0.00000,C#,0.1000,-5.205,Major,0.0277,78.995,4/4,0.6440
91560,Hip-Hop,49,0.02760,0.484,169286,0.54100,0.00000,G,0.1380,-8.701,Major,0.0370,139.981,4/4,0.1370
160626,Reggaeton,39,0.03550,0.707,209288,0.84200,0.00007,B,0.0793,-3.004,Minor,0.1720,101.984,4/4,0.7330
210855,World,32,0.98700,0.270,175773,0.00157,0.88200,C#,0.1090,-31.359,Major,0.0438,62.892,3/4,0.0798
56580,R&B,55,0.44500,0.457,234802,0.42500,0.00000,E,0.0714,-10.500,Minor,0.2430,201.472,4/4,0.4980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42466,Folk,59,0.23800,0.709,390649,0.36900,0.16800,E,0.0704,-13.104,Minor,0.0488,102.010,3/4,0.3730
206348,Soundtrack,25,0.95500,0.354,56000,0.10600,0.97500,C#,0.1130,-22.190,Major,0.0387,139.863,4/4,0.0322
166062,Rock,88,0.00671,0.605,187147,0.82200,0.13400,C,0.1470,-4.833,Major,0.0438,167.997,4/4,0.2880
17027,Dance,54,0.02510,0.692,250387,0.90800,0.00000,A,0.0933,-5.364,Major,0.0677,124.042,4/4,0.5130


# X,y split

In [ ]:
col_name = "genre"
y =  df2[col_name]
X =  df2.drop([col_name], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# scaling

In [ ]:
col_num = X_train.select_dtypes(include = np.number).columns

In [ ]:
col_name = "key"
cat_list = ["A", "A#", "B","B#","C","C#","D","D#","E","E#","F","F#","G","G#"]
OrdEncod = OrdinalEncoder(categories = [cat_list ])
X_train[col_name] = OrdEncod.fit_transform(X_train[col_name].values.reshape(-1, 1))
X_test[col_name] = OrdEncod.transform(X_test[col_name].values.reshape(-1, 1))

X_train

,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
19965,55,0.0589,0.589,258947,0.727,0.000000,1.0,0.134,-6.622,Minor,0.0314,95.001,4/4,0.356
49298,34,0.0938,0.586,350200,0.719,0.001220,6.0,0.115,-5.358,Major,0.0245,87.506,4/4,0.772
200338,33,0.6400,0.437,177400,0.321,0.849000,4.0,0.102,-14.768,Major,0.0322,123.281,4/4,0.369
15632,60,0.1840,0.746,258453,0.691,0.025900,8.0,0.205,-7.533,Minor,0.0403,119.982,4/4,0.888
47879,43,0.0298,0.435,581733,0.797,0.034600,4.0,0.043,-7.741,Major,0.0563,98.830,4/4,0.711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21934,49,0.6650,0.678,187213,0.415,0.000000,4.0,0.106,-11.561,Major,0.1390,114.987,4/4,0.333
149036,39,0.9600,0.479,268625,0.146,0.000002,10.0,0.116,-9.482,Major,0.0331,123.634,4/4,0.120
225592,56,0.1390,0.803,221234,0.521,0.008990,4.0,0.081,-7.775,Major,0.0276,99.460,4/4,0.969
109450,70,0.3930,0.683,290791,0.695,0.001240,5.0,0.124,-6.798,Minor,0.0574,135.337,4/4,0.406


In [ ]:
col_name = "time_signature"
cat_list = ['0/4', '1/4', '2/4', '3/4', '4/4','5/4']
OrdEncod = OrdinalEncoder(categories = [cat_list ])
X_train[col_name] = OrdEncod.fit_transform(X_train[col_name].values.reshape(-1, 1))
X_test[col_name] = OrdEncod.transform(X_test[col_name].values.reshape(-1, 1))

X_train

,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
19965,55,0.0589,0.589,258947,0.727,0.000000,1.0,0.134,-6.622,Minor,0.0314,95.001,4.0,0.356
49298,34,0.0938,0.586,350200,0.719,0.001220,6.0,0.115,-5.358,Major,0.0245,87.506,4.0,0.772
200338,33,0.6400,0.437,177400,0.321,0.849000,4.0,0.102,-14.768,Major,0.0322,123.281,4.0,0.369
15632,60,0.1840,0.746,258453,0.691,0.025900,8.0,0.205,-7.533,Minor,0.0403,119.982,4.0,0.888
47879,43,0.0298,0.435,581733,0.797,0.034600,4.0,0.043,-7.741,Major,0.0563,98.830,4.0,0.711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21934,49,0.6650,0.678,187213,0.415,0.000000,4.0,0.106,-11.561,Major,0.1390,114.987,4.0,0.333
149036,39,0.9600,0.479,268625,0.146,0.000002,10.0,0.116,-9.482,Major,0.0331,123.634,4.0,0.120
225592,56,0.1390,0.803,221234,0.521,0.008990,4.0,0.081,-7.775,Major,0.0276,99.460,4.0,0.969
109450,70,0.3930,0.683,290791,0.695,0.001240,5.0,0.124,-6.798,Minor,0.0574,135.337,4.0,0.406


In [ ]:
df2["genre"].unique()

array(['R&B', 'Hip-Hop', 'Reggaeton', 'World', 'Rap', 'Soundtrack',
       'Reggae', 'Alternative', 'Indie', "Children's Music", 'Comedy',
       'Electronic', 'Blues', 'Rock', 'Soul', 'Opera', 'Dance', 'Ska',
       'Movie', 'Children’s Music', 'Folk', 'Pop', 'Country', 'Jazz',
       'Classical', 'Anime', 'A Capella'], dtype=object)

In [ ]:
'''col_name = "genre"
cat_list = ['Movie', 'R&B', 'A Capella', 'Alternative', 'Country', 'Dance',
       'Electronic', 'Anime', 'Folk', 'Blues', 'Opera', 'Hip-Hop',
       "Children's Music", 'Children’s Music', 'Rap', 'Indie',
       'Classical', 'Pop', 'Reggae', 'Reggaeton', 'Jazz', 'Rock', 'Ska',
       'Comedy', 'Soul', 'Soundtrack', 'World']'''
col_name = "mode"
cat_list = ['Major', 'Minor']
OrdEncod = OrdinalEncoder(categories = [cat_list ])
X_train[col_name] = OrdEncod.fit_transform(X_train[col_name].values.reshape(-1, 1))
X_train[col_name] = X_train[col_name].astype(int)
X_test[col_name] = OrdEncod.transform(X_test[col_name].values.reshape(-1, 1))

X_test[col_name] = X_test[col_name].astype(int)
X_train

,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
19965,55,0.0589,0.589,258947,0.727,0.000000,1.0,0.134,-6.622,1,0.0314,95.001,4.0,0.356
49298,34,0.0938,0.586,350200,0.719,0.001220,6.0,0.115,-5.358,0,0.0245,87.506,4.0,0.772
200338,33,0.6400,0.437,177400,0.321,0.849000,4.0,0.102,-14.768,0,0.0322,123.281,4.0,0.369
15632,60,0.1840,0.746,258453,0.691,0.025900,8.0,0.205,-7.533,1,0.0403,119.982,4.0,0.888
47879,43,0.0298,0.435,581733,0.797,0.034600,4.0,0.043,-7.741,0,0.0563,98.830,4.0,0.711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21934,49,0.6650,0.678,187213,0.415,0.000000,4.0,0.106,-11.561,0,0.1390,114.987,4.0,0.333
149036,39,0.9600,0.479,268625,0.146,0.000002,10.0,0.116,-9.482,0,0.0331,123.634,4.0,0.120
225592,56,0.1390,0.803,221234,0.521,0.008990,4.0,0.081,-7.775,0,0.0276,99.460,4.0,0.969
109450,70,0.3930,0.683,290791,0.695,0.001240,5.0,0.124,-6.798,1,0.0574,135.337,4.0,0.406


In [ ]:
list(col_num)

['popularity',
 'acousticness',
 'danceability',
 'duration_ms',
 'energy',
 'instrumentalness',
 'liveness',
 'loudness',
 'speechiness',
 'tempo',
 'valence']

In [ ]:
scaler = StandardScaler()
X_train[col_num] = scaler.fit_transform(X_train[col_num])
X_test[col_num] = scaler.fit_transform(X_test[col_num])
X_train

,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
19965,0.760263,-0.869805,0.190130,0.198212,0.589397,-0.487175,1.0,-0.408556,0.489027,1,-0.482720,-0.728590,4.0,-0.377531
49298,-0.396080,-0.771383,0.173940,0.957321,0.559032,-0.483128,6.0,-0.504746,0.700536,0,-0.519822,-0.970783,4.0,1.229248
200338,-0.451144,0.768971,-0.630201,-0.480156,-0.951618,2.329332,4.0,-0.570560,-0.874065,0,-0.478418,0.185248,4.0,-0.327319
15632,1.035583,-0.517007,1.037446,0.194102,0.452755,-0.401253,8.0,-0.049110,0.336587,1,-0.434863,0.078644,4.0,1.677292
47879,0.099496,-0.951871,-0.640995,2.883382,0.855089,-0.372392,4.0,-0.869255,0.301782,0,-0.348828,-0.604860,4.0,0.993638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21934,0.429879,0.839475,0.670456,-0.398524,-0.594832,-0.487175,4.0,-0.550309,-0.337429,0,0.095863,-0.082764,4.0,-0.466367
149036,-0.120760,1.671412,-0.403530,0.278720,-1.615849,-0.487168,10.0,-0.499683,0.010456,0,-0.473579,0.196654,4.0,-1.289069
225592,0.815327,-0.643913,1.345070,-0.115513,-0.192498,-0.457351,4.0,-0.676875,0.296092,0,-0.503153,-0.584502,4.0,1.990150
109450,1.586222,0.072400,0.697440,0.463113,0.467938,-0.483061,5.0,-0.459182,0.459577,1,-0.342914,0.574824,4.0,-0.184408


## X y chois + train_testsplit

## Execute the following tasks:

    Build a classification or regression model with this dataset
    
     - Evaluate the performance of your model using accuracy score, RMSE or R_squared
     - Apply 3 ensemble techniques of your choice on the dataset
     - Compare the obtained results with your individual model


In [ ]:
import warnings
from sklearn.exceptions import DataConversionWarning


In [ ]:
clf_1 = KNeighborsClassifier()

clf_1.fit(X_train, y_train)
y_pred = clf_1.predict(X_test)
print(clf_1.__class__.__name__, accuracy_score(y_test, y_pred))

KNeighborsClassifier 0.31645


# Hard voting

In [ ]:
clf_1 = KNeighborsClassifier()
clf_2 = GaussianNB()
clf_3 = DecisionTreeClassifier()


# Create voting classifier
voting_ens = VotingClassifier(estimators=[('knn', clf_1), ('lr', clf_2), ('dt', clf_3)], voting='hard')


# Fit and predict with the models and ensemble
for clf in (clf_1, clf_2, clf_3, voting_ens):
   clf.fit(X_train, y_train)
   y_pred = clf.predict(X_test)

   print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
   print("\n")

KNeighborsClassifier 0.31645


GaussianNB 0.30865


DecisionTreeClassifier 0.29415


VotingClassifier 0.326




### StackingClassifier

In [ ]:
#from mlxtend.classifier import StackingClassifier
from sklearn.ensemble import StackingClassifier

# Initialize individual models

clf_1 = KNeighborsClassifier()
clf_2 = GaussianNB()
clf_3 = DecisionTreeClassifier()

estimators = [
     ('DTC', DecisionTreeClassifier()),
     ('Gaussian', LogisticRegression()),
     ('KNc',KNeighborsClassifier())
]



# Initialize meta-model
clf_meta = LogisticRegression()


# Create stacking classifier
clf_stack = StackingClassifier(
                              estimators=estimators,
                              final_estimator=LogisticRegression(),
                              )


# Fit and predict with the models and ensemble

for clf in (clf_1, clf_2, clf_3, clf_stack):
   print(clf)
   clf.fit(X_train, y_train)
   y_pred = clf.predict(X_test)

   print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
   print("\n")

KNeighborsClassifier()
KNeighborsClassifier 0.31645


GaussianNB()
GaussianNB 0.30865


DecisionTreeClassifier()
DecisionTreeClassifier 0.2971


StackingClassifier(estimators=[('DTC', DecisionTreeClassifier()),
                               ('Gaussian', LogisticRegression()),
                               ('KNc', KNeighborsClassifier())],
                   final_estimator=LogisticRegression())


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

StackingClassifier 0.419




In [ ]:
!pip install mlxtend

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sklearn.ensemble import BaggingClassifier


# Initialize weak model

base_model = DecisionTreeClassifier(max_depth=3)


# Create the bagging classifier

clf_bagging = BaggingClassifier(base_estimator=base_model, n_estimators=1000, oob_score=True)

clf_bagging.fit(X_train, y_train)


# Check oob score

print("oob_score_: ", clf_bagging.oob_score_)

# Compare with test set

pred = clf_bagging.predict(X_test)

print("accuracy_score: ", accuracy_score(y_test, pred))

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


oob_score_:  0.2022625
accuracy_score:  0.19785
